<a href="https://colab.research.google.com/github/lokeshtalamala1/Deep_Learning_Assignment_1/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#initializing my project and entity in wandb using its atributes
#Deep_Learning_Assignment_1
#cs24m023-indian-institute-of-technology-madras

!pip install wandb tensorflow
import wandb
wandb.init(project='Deep_Learning_Assignment_1', entity='cs24m023-indian-institute-of-technology-madras')


In [3]:
#import numpy ,fashion_mnist and load data

from keras.datasets import fashion_mnist
import numpy as np

((x_train,y_train),(x_test,y_test)) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
import matplotlib.pyplot as plt

In [5]:
#example image from each class from given data

#QUESTION 1

images = []
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for class_label in range(len(classes)):
    index = next(i for i, y in enumerate(y_train) if y == class_label)
    images.append(wandb.Image(x_train[index], caption=classes[class_label]))

wandb.log({"example image from each class": images})
plt.show()
wandb.finish()

In [6]:
#QUESTION 2 AND QUESTION 3
#creating a class with a layer as an object and parametes to the layer are :
    #input dimensions
    #activation_function
    #optimizer_function
    #weight initialization

#FORWARD PROPAGATE AND BACKWARD PROPAGATE
#feedforwardneuralnetwork for calculating all the layers over the model and loss functions are calculated
#with input parameters optimizer,activation function weight initialization
#number of epochs and size of each layer ,learnig rate theeta

import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# -------------------- FeedForwardNN Class --------------------
class FeedForwardNN:
    def __init__(self, input_size=None, hidden_nodes=None, output_size=None, layer_dims=None,
                 activation_hidden='relu', init_type='xavier'):
        """
        Initializes the neural network.

        You can specify the architecture in two ways:
          1. Directly pass a list of layer dimensions as 'layer_dims'.
          2. Or pass the input size, list of hidden layer node counts, and output size.
        """
        if layer_dims is not None:
            self.layer_dims = layer_dims
        else:
            if input_size is None or hidden_nodes is None or output_size is None:
                raise ValueError("Either layer_dims or input_size, hidden_nodes, and output_size must be provided")
            self.layer_dims = [input_size] + hidden_nodes + [output_size]

        self.L = len(self.layer_dims) - 1  # Number of layers excluding the input layer
        self.init_type = init_type
        self.activation_hidden = activation_hidden
        self.parameters = self.initialize_parameters(self.layer_dims, init_type)

    # ---- Activation Functions and Their Gradients ----
    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    def dsigmoid(z):
        s = FeedForwardNN.sigmoid(z)
        return s * (1 - s)

    def tanh(z):
        return np.tanh(z)

    def dtanh(z):
        return 1 - np.tanh(z)**2

    def relu(z):
        return np.maximum(0, z)

    def drelu(z):
        return np.array(z > 0, dtype=float)

    def softmax(z):
        z_exp = np.exp(z - np.max(z, axis=0, keepdims=True))
        return z_exp / np.sum(z_exp, axis=0, keepdims=True)

    def dsoftmax(z):
        s = FeedForwardNN.softmax(z.reshape(-1, 1))
        jacobian = np.diagflat(s) - np.dot(s, s.T)
        return jacobian

    def get_activation_function(self):
        if self.activation_hidden.lower() == 'sigmoid':
            return FeedForwardNN.sigmoid, FeedForwardNN.dsigmoid
        elif self.activation_hidden.lower() == 'tanh':
            return FeedForwardNN.tanh, FeedForwardNN.dtanh
        elif self.activation_hidden.lower() == 'relu':
            return FeedForwardNN.relu, FeedForwardNN.drelu
        else:
            raise ValueError("Unsupported activation function. Choose from 'sigmoid', 'tanh', or 'relu'.")

    # ---- Parameter Initialization ----
    def initialize_parameters(self, layer_dims, init_type='random'):
        parameters = {}
        L = len(layer_dims)
        for l in range(1, L):
            if init_type.lower() == 'random':
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])
            elif init_type.lower() == 'xavier':
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(1.0 / layer_dims[l-1])
            else:
                raise ValueError("init_type must be 'random' or 'xavier'")
            parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        return parameters

    # ---- Forward Propagation ----
    def forward_propagation(self, X):
        caches = []
        A = X
        activation_func, _ = self.get_activation_function()
        # Process hidden layers
        for l in range(1, self.L):
            W = self.parameters['W' + str(l)]
            b = self.parameters['b' + str(l)]
            Z = np.dot(W, A) + b
            A = activation_func(Z)
            caches.append((Z, A))
        # Output layer with softmax activation
        W = self.parameters['W' + str(self.L)]
        b = self.parameters['b' + str(self.L)]
        Z = np.dot(W, A) + b
        A_final = FeedForwardNN.softmax(Z)
        caches.append((Z, A_final))
        return A_final, caches

    # ---- Backpropagation ----
    def back_propagation(self, X, Y, caches, weight_decay=0.0):
        grads = {}
        m = X.shape[1]
        L = self.L
        _, activation_grad = self.get_activation_function()
        # Output layer gradient (using softmax + cross-entropy)
        ZL, AL = caches[-1]
        dZL = AL - Y
        WL_prev = self.parameters['W' + str(L)]
        A_prev = caches[-2][1] if L > 1 else X
        grads["dW" + str(L)] = (np.dot(dZL, A_prev.T) / m) + weight_decay * self.parameters['W' + str(L)]
        grads["db" + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m
        dA_prev = np.dot(WL_prev.T, dZL)
        # Backprop for hidden layers
        for l in reversed(range(1, L)):
            Z, A = caches[l-1]
            dZ = dA_prev * activation_grad(Z)
            A_prev = X if l == 1 else caches[l-2][1]
            grads["dW" + str(l)] = (np.dot(dZ, A_prev.T) / m) + weight_decay * self.parameters['W' + str(l)]
            grads["db" + str(l)] = np.sum(dZ, axis=1, keepdims=True) / m
            if l > 1:
                W_prev = self.parameters['W' + str(l)]
                dA_prev = np.dot(W_prev.T, dZ)
        return grads

    # ---- Compute Loss and Accuracy ----
    def compute_loss_and_accuracy(self, X, Y):
        A, _ = self.forward_propagation(X)
        m = X.shape[1]
        loss = -np.sum(Y * np.log(A + 1e-8)) / m
        predictions = np.argmax(A, axis=0)
        labels = np.argmax(Y, axis=0)
        accuracy = np.mean(predictions == labels)
        return loss, accuracy

    # ---- Training Step (Mini-Batch) ----
    def train_step(self, X_batch, Y_batch, optimizer, weight_decay, t=1):
        A_final, caches = self.forward_propagation(X_batch)
        m = X_batch.shape[1]
        loss = -np.sum(Y_batch * np.log(A_final + 1e-8)) / m
        grads = self.back_propagation(X_batch, Y_batch, caches, weight_decay=weight_decay)
        self.parameters = optimizer.update_parameters(self.parameters, grads, t)
        return loss

    # ---- Training Loop with Mini-Batches ----
    def train(self, X_train, Y_train, X_val, Y_val, optimizer, epochs=10, batch_size=32, weight_decay=0.0):
        m = X_train.shape[1]
        t = 1  # time step for optimizers (e.g., Adam)
        for epoch in range(1, epochs + 1):
            permutation = np.random.permutation(m)
            X_shuffled = X_train[:, permutation]
            Y_shuffled = Y_train[:, permutation]
            epoch_loss = 0
            num_batches = m // batch_size
            for i in range(num_batches):
                start = i * batch_size
                end = start + batch_size
                X_batch = X_shuffled[:, start:end]
                Y_batch = Y_shuffled[:, start:end]
                loss = self.train_step(X_batch, Y_batch, optimizer, weight_decay, t)
                epoch_loss += loss
                t += 1
            epoch_loss /= num_batches
            train_loss, train_acc = self.compute_loss_and_accuracy(X_train, Y_train)
            val_loss, val_acc = self.compute_loss_and_accuracy(X_val, Y_val)
            print(f"Epoch {epoch:03d}: Epoch Loss = {epoch_loss:.4f} | Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} | Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
        # Optionally, metrics can be returned or logged externally.


# -------------------- Optimizers Class --------------------
class Optimizers:
    def __init__(self, method='sgd', learning_rate=0.01, momentum=0.9, beta1=0.9, beta2=0.999, epsilon=1e-8, decay=0.0):
        """
        Initializes the optimizer.

        Supported methods: 'sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam'
        """
        self.method = method.lower()
        self.lr = learning_rate
        self.momentum = momentum
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.decay = decay
        self.v = {}
        self.s = {}

    def initialize_state(self, parameters):
        for key in parameters:
            self.v[key] = np.zeros_like(parameters[key])
            self.s[key] = np.zeros_like(parameters[key])

    def update_parameters(self, parameters, grads, t=1):
        if not self.v or not self.s:
            self.initialize_state(parameters)
        updated_parameters = {}
        for key in parameters.keys():
            grad = grads.get("d" + key, None)
            if grad is None:
                updated_parameters[key] = parameters[key]
                continue
            if self.method == 'sgd':
                update = self.lr * grad
            elif self.method == 'momentum':
                self.v[key] = self.momentum * self.v[key] + self.lr * grad
                update = self.v[key]
            elif self.method == 'nesterov':
                v_prev = self.v[key].copy()
                self.v[key] = self.momentum * self.v[key] + self.lr * grad
                update = self.momentum * v_prev + (1 + self.momentum) * self.lr * grad
            elif self.method == 'rmsprop':
                self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (grad ** 2)
                update = self.lr * grad / (np.sqrt(self.s[key]) + self.epsilon)
            elif self.method == 'adam':
                self.v[key] = self.beta1 * self.v[key] + (1 - self.beta1) * grad
                self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (grad ** 2)
                v_corrected = self.v[key] / (1 - self.beta1**t)
                s_corrected = self.s[key] / (1 - self.beta2**t)
                update = self.lr * v_corrected / (np.sqrt(s_corrected) + self.epsilon)
            elif self.method == 'nadam':
                self.v[key] = self.beta1 * self.v[key] + (1 - self.beta1) * grad
                self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (grad ** 2)
                v_corrected = self.v[key] / (1 - self.beta1**t)
                s_corrected = self.s[key] / (1 - self.beta2**t)
                update = self.lr * (self.beta1 * v_corrected + (1 - self.beta1) * grad / (1 - self.beta1**t)) / (np.sqrt(s_corrected) + self.epsilon)
            else:
                raise ValueError("Unsupported optimization method.")
            updated_parameters[key] = parameters[key] - update
        return updated_parameters


